In [38]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [39]:
def training_data_split(X_observed_clean, X_estimated_clean_mean):
    X_train_estimated = X_estimated_clean_mean[:int(X_estimated_clean_mean.shape[0] * 3 / 4)]
    X_valid = X_estimated_clean_mean[int(X_estimated_clean_mean.shape[0] * 3 / 4):int(X_estimated_clean_mean.shape[0] * 9 / 10)]
    X_test = X_estimated_clean_mean[int(X_estimated_clean_mean.shape[0] * 9 / 10):]

    X_train = pd.concat([X_observed_clean, X_train_estimated])
    return X_train, X_valid, X_test

#A function which takes the mean out of every 4th column and saves it on the time on the time of the 4th. Makes it so it is every hour.
#TODO: Should be swapped for Gustavs code!
def mean_df(df):
    # Assuming df is your DataFrame and 'date_forecast' is your date column
    # Making a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()

    # Step 1: Keeping every 4th row in the date column
    date_column = df_copy['date_forecast'].iloc[::4]

    # Step 2: Creating a grouping key
    grouping_key = np.floor(np.arange(len(df_copy)) / 4)

    # Step 3: Group by the key and calculate the mean, excluding the date column
    averaged_data = df_copy.drop(columns=['date_forecast']).groupby(grouping_key).mean()

    # Step 4: Reset index and merge the date column
    averaged_data.reset_index(drop=True, inplace=True)
    averaged_data['date_forecast'] = date_column.values
    return averaged_data

#Removes all features from a df except selected_features
def clean_df(df, selected_features):
    return df[selected_features]

  #Scales all the feature value in a way they take a simmilar range
def scale_df(df):
    scaler = StandardScaler()
    df = scaler.fit_transform(df)
    return df

#Function which resizes the training data such that only the rows with the same date and time for weather is kept.
#X_train is either observed or forcasted weather and y_train is how much energy is produced.
#y_features are a list containing the column names of y_train
#X_date_feature is the feature name which the date and time for the weather is savew. This will probably always be "date_forecast" and may be changed
def resize_training_data(X_train, y_train):
    y_features = y_train.columns.tolist()
    X_date_feature = "date_forecast"

    merged = pd.merge(X_train, y_train,left_on=X_date_feature, right_on='time', how='inner')
    y_train_resized = merged[y_features]
    columns_to_drop = y_features + [X_date_feature]
    X_train_resized = merged.drop(columns = columns_to_drop)
    return X_train_resized, y_train_resized

In [40]:
selected_features = ["date_forecast", "direct_rad:W", "clear_sky_rad:W" , "diffuse_rad:W","is_in_shadow:idx", "relative_humidity_1000hPa:p"]

In [41]:
y = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/train_targets.parquet')
X_estimated= pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/X_train_estimated.parquet')
X_observed= pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/X_train_observed.parquet')
X_test_real = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/ML/C/X_test_estimated.parquet')
y = y.dropna()

In [42]:
X_estimated_clean = clean_df(X_estimated, selected_features)
X_observed_clean = clean_df(X_observed, selected_features)
X_test_real_clean = clean_df(X_test_real, selected_features)
X_estimated_clean_mean = mean_df(X_estimated_clean)
X_test_real_clean_mean = mean_df(X_test_real_clean)

In [43]:
X_train, X_valid, X_test = training_data_split(X_observed_clean, X_estimated_clean_mean)

In [44]:
X_train, y_train = resize_training_data(X_train, y)
X_valid, y_valid = resize_training_data(X_valid, y)
X_test, y_test = resize_training_data(X_test, y)

In [45]:
#model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 10)
#model.fit(X_train, y_train["pv_measurement"])


In [46]:
#predictions = model.predict(X_test)

In [47]:
#mae = mean_absolute_error(y_test["pv_measurement"], predictions)
#print(f"Mean Absolute Error: {mae}")

In [48]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'n_estimators': [100, 200, 500],
    'objective': ['reg:squarederror']
}

# Initialize XGBoost Regressor
xg_reg = xgb.XGBRegressor()

# Initialize Grid Search
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3, verbose=2)

# Fit the model
grid_search.fit(X_train, y_train["pv_measurement"])

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

# Train a new XGBoost model with the best parameters
best_xg_model = xgb.XGBRegressor(**best_params)
best_xg_model.fit(X_train, y_train["pv_measurement"])
# Calculate the Mean Absolute Error
#mae = mean_absolute_error(y_test["pv_measurement"], predictions)
#print(f"Mean Absolute Error of the best model: {mae}")
#Best parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'objective': 'reg:squarederror', 'subsample': 0.7}

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.5; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimators=100, objective=reg:squarederror, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.5, learning_rate=0.01, max_depth=3, n_estimat

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
# Make predictions
predictions = best_xg_model.predict(X_test_real_clean_mean.drop(columns = ["date_forecast"]))
np.save('X_test_c.npy', predictions)